In [112]:
import pandas as pd 
import numpy as np
import string
import json
import pickle
import math

# Phase1

In [113]:


#word_freq_dict will save the frequency of each word in all of the documents
number_of_all_docs = 10
word_freq_dict = {}
#inverted_index_dict will save the inverted index for each word, 
#and also the positions of each word in all the documents
inverted_index_dict = {}
#This dictionary will be like:
"""
inverted_index_dict={
    "word": {"docs": {
                    "doc_id":{
                            "positions": [],
                            "tf": int
                    }
             },
            "df": int
    }
}



"""


#This dictionary will save docs as keys and terms of each doc as their values 
doc_term_dict = {}
"""
doc_term_dict ={
                
                "doc_number": [term1, term2, ..., term_n]

}

"""


stop_words = []

In [114]:
dic1 = {"1":1, "2":22, "3":333}
# # if "4" not in dic1.keys():
# #     dic1["4"] = 4444
# dic1
# stringam = "سلام امیررضا خوبی داداش من خوبم"
f = open("./stop_words.txt", "r")

stop_words_string = f.read()
stop_words_list = stop_words_string.split()
len(stop_words_list)

150

In [115]:
#This method will do some normalization on text files. 
# def normalizer (words_list):
    

In [116]:
#This method will return the positions of a word in a text file (the input is tokens list)
def word_positions (word, tokenized_text):
    positions = [i for i, x in enumerate(tokenized_text) if x == word]
    return positions

#This funcntions will make a list of unique words in a words_list
def different_words (words_list):
    
    different_words_list = []
    for word in words_list:
        if word not in different_words_list:
            different_words_list.append(word)
            
    return different_words_list


In [117]:
#This method will update the inverted index by processing new document
def update_inverted_index (text_file_number, inverted_index_dict, word_freq_dict):
    #Reading the text file 
    
    text_file_name = str(text_file_number)
    file_path = "./sampleDoc/" + text_file_name + ".txt"
    f = open(file_path, "r")

    text = f.read()

    #Remmoving punctuations from the text string
    result = text.translate(str.maketrans('','', string.punctuation))

    #Removin نیم فاصله from the text string
    result = result.replace("\u200c", " ")
#     resutl = result.replace("\u200b", "")
    
    #Splitting the string into seperate words (tokenizing)
    words_list = result.split()
    
    different_words_list = different_words (words_list)
    
    #This loop will update the words frequency dictionary
    for word in words_list:
        if word not in word_freq_dict.keys():
            word_freq_dict[word] = 1
        else:
            word_freq_dict[word] += 1
    
    #This loop will update the inverted index dictionary 
    for word in different_words_list:
        word_positions_list = word_positions (word, words_list) 
        if word not in inverted_index_dict.keys():
            inverted_index_dict[word] = {"docs":{}, "df": 1}
            inverted_index_dict[word]["docs"][text_file_name] = {"positions": word_positions_list, "tf":len(word_positions_list)}
        else:
            inverted_index_dict[word]["docs"][text_file_name] = {"positions": word_positions_list, "tf":len(word_positions_list)}
            inverted_index_dict[word]["df"] += 1
            

In [118]:
# update_inverted_index (1, inverted_index_dict, word_freq_dict)

inverted_index_dict

{}

In [119]:
for i in range (1, number_of_all_docs + 1):
    update_inverted_index (i, inverted_index_dict, word_freq_dict)

inverted_index_dict["گزارش"]

{'docs': {'1': {'positions': [8], 'tf': 1},
  '3': {'positions': [16], 'tf': 1},
  '5': {'positions': [13], 'tf': 1},
  '7': {'positions': [23], 'tf': 1},
  '9': {'positions': [0, 40], 'tf': 2},
  '10': {'positions': [55], 'tf': 1}},
 'df': 6}

In [120]:
#This method will save a dictionary with given name into a pickle file
inverted_index_dict
def save_dict (input_dict, file_name):
    
    saved_file = open(file_name, "wb")
    pickle.dump(input_dict, saved_file)
    saved_file.close()

save_dict(inverted_index_dict, "inverted_index.pkl")


In [121]:
#We load the saved pickle file 

def load_dict (file_name):

    saved_file = open(file_name, "rb")
    output = pickle.load(saved_file)
    return output

loaded_dict = load_dict("inverted_index.pkl")
loaded_dict

{'نصوریان': {'docs': {'1': {'positions': [0], 'tf': 1}}, 'df': 1},
 'سرمربی': {'docs': {'1': {'positions': [1, 37], 'tf': 2}}, 'df': 1},
 'تراکتور': {'docs': {'1': {'positions': [2, 14, 46, 80], 'tf': 4}}, 'df': 1},
 'از': {'docs': {'1': {'positions': [3, 24], 'tf': 2},
   '3': {'positions': [69, 120, 203], 'tf': 3},
   '4': {'positions': [190], 'tf': 1},
   '5': {'positions': [59], 'tf': 1},
   '6': {'positions': [45, 61, 73, 194], 'tf': 4},
   '9': {'positions': [19, 44, 108, 130], 'tf': 4},
   '10': {'positions': [3, 60, 88, 167], 'tf': 4}},
  'df': 7},
 'سمتش': {'docs': {'1': {'positions': [4, 25], 'tf': 2}}, 'df': 1},
 'برکنار': {'docs': {'1': {'positions': [5, 26], 'tf': 2}}, 'df': 1},
 'شد': {'docs': {'1': {'positions': [6, 27], 'tf': 2},
   '2': {'positions': [4, 170], 'tf': 2},
   '3': {'positions': [84], 'tf': 1},
   '4': {'positions': [88, 296, 429], 'tf': 3},
   '9': {'positions': [102, 134], 'tf': 2}},
  'df': 5},
 'به': {'docs': {'1': {'positions': [7, 58, 64], 'tf': 3},


In [122]:
#First we make an empty list for each document in doc_term_dict dictionary 
for i in range (1, number_of_all_docs + 1):
    doc_term_dict[str(i)] = []
    
#Then we will the doc_term_dict dictionary with each doc's terms    
for term in inverted_index_dict:
    
    for doc in inverted_index_dict[term]["docs"].keys():
        doc_term_dict[doc].append(term)

doc_term_dict


{'1': ['نصوریان',
  'سرمربی',
  'تراکتور',
  'از',
  'سمتش',
  'برکنار',
  'شد',
  'به',
  'گزارش',
  'تابناک',
  'ورزشی،',
  'با',
  'تصمیم',
  'مدیران',
  'و',
  'بدلیل',
  'نتایج',
  'ضعیف',
  'این',
  'تیم',
  'پرستاره',
  'علیرضا',
  'منصوریان',
  'زمستان',
  'تبریز',
  'را',
  'هم',
  'ندید',
  'علت',
  'برکناری',
  'کسب',
  'بود',
  'هدایت',
  'توانسته',
  'تنها',
  'یک',
  'پیروزی',
  'در',
  '۵',
  'هفته',
  'نخست',
  'لیگ',
  'بیستم',
  'دست',
  'بیاورد',
  'نام',
  'ساکت',
  'الهامی',
  'عنوان',
  'جدی',
  'ترین',
  'گزینه',
  'جانشینی',
  'مطرح',
  'شده',
  'است',
  'فصل',
  'گذشته',
  'قهرمان',
  'جام',
  'حذفی',
  'شود'],
 '2': ['شد',
  'به',
  'با',
  'و',
  'هم',
  'یک',
  'در',
  'شاخص',
  'بورس',
  'بازهم',
  'قرمز',
  'کل',
  '5246',
  'واحد',
  'کاهش',
  'معادل',
  'منفی',
  '035',
  'درصد',
  'میلیون',
  '507',
  'هزار',
  '645',
  'واحد،',
  'قیمت',
  'وزنی',
  'ارزشی',
  '1372',
  '394',
  '278',
  'وزن',
  '1688',
  'افزایش',
  '038',
  '444',
  '424',
  '1105',

In [123]:


# frequent_words = []

# for word in word_freq_dict:
#     if word_freq_dict[word] > 30:
#         frequent_words.append(word)

# frequent_words

output["تابناک"]["docs"].keys()

dict_keys(['1', '3', '5', '7', '8', '10'])

In [124]:
#This method will sort the documents based on their appearance in results
def sort_docs(query_response): 
    
    query_response_keys = list(query_response.keys())
    query_response_values = list(query_response.values())
    sort_docs_dict = {}
    for doc_list in query_response_values:
        for doc in doc_list:
            if doc not in sort_docs_dict.keys():
                sort_docs_dict[doc] = 1
            else:
                sort_docs_dict[doc] += 1
                
    sort_docs_dict = {k: v for k, v in sorted(sort_docs_dict.items(), key=lambda item: item[1])}
    
    return sort_docs_dict
    
    
#     return list(set(query_response_values[0]) & set(query_response_values[1])) 
   
    

#This method will process the user's query and prints response for him
def query ():
    
    query_response = {}
    query = input()
    query_tokens = query.split()
    
    #This counter is to check whether there is any match with this query or not
    counter = 0
    
    for token in query_tokens:
        if token in output.keys():
            counter += 1
            query_response[token] = list(output[token]["docs"].keys())
            
    if counter == 0:
        print("No match found!")
    else:
#         print("salam")
#         print(query_response)
        ans = sort_docs(query_response) 
        print(ans)
    


In [125]:
# query()
output['گزارش']


{'docs': {'1': {'positions': [8], 'tf': 1},
  '3': {'positions': [16], 'tf': 1},
  '5': {'positions': [13], 'tf': 1},
  '7': {'positions': [23], 'tf': 1},
  '9': {'positions': [0, 40], 'tf': 2},
  '10': {'positions': [55], 'tf': 1}},
 'df': 6}

In [126]:
query()

گزارش تابناک
{'9': 1, '8': 1, '1': 2, '3': 2, '5': 2, '7': 2, '10': 2}


# Phase2

In [135]:
#This method will calculate the tf-idf weight for given term, and document
#Instead of storing the weights we will calculate them based on tf and idf, because it will occupy lower space 
def calculate_weight (term, document_number, inverted_index_dict):
    
#     print(term)
#     print(document_number)
#     print(inverted_index_dict[term])
    # !!! document number is a string e.g. "x"
    if document_number not in inverted_index_dict[term]["docs"].keys():
        tf_td = 0
    else:
        tf_td = inverted_index_dict[term]["docs"][document_number]['tf']
        
    idf_t = math.log10(number_of_all_docs/inverted_index_dict[term]['df'])
    
    if tf_td > 0:
        w_td = (1 + math.log10(tf_td)) * idf_t
    else:
        w_td = 0
    
    return w_td

In [136]:
score = calculate_weight("گزارش", "2", inverted_index_dict)
score

0

In [137]:
#This method will calculate document length for vector normalization
def calculate_doc_vector_length(document):
    
    sum = 0
    
    for term in doc_term_dict[document]:
        
        w_term = calculate_weight(term, document, inverted_index_dict)
        w2 = math.pow(w_term, 2)
        sum += w2
    
    length = math.sqrt(sum)
    
    return length

def calculate_tf_in_query(normalized_query_term_list):
    
    tf_dict = {}
    
    for term in normalized_query_term_list:
        
        if term in tf_dict:
            tf_dict[term] += 1
        
        else:
            tf_dict[term] = 1
            
    return tf_dict

#This method will calculate query length for vector normalization
def calculate_query_vector_length(normalized_query_term_list):
    
    tf_dict = calculate_tf_in_query(normalized_query_term_list)
    query_terms_weight = {}
    sum = 0
    
    for term in normalized_query_term_list: 
        
        tf_tq = tf_dict[term]
        idf_t = math.log10(number_of_all_docs/inverted_index_dict[term]['df'])
        
        w_tq = (1 + math.log10(tf_tq)) * idf_t
        query_terms_weight[term] = w_tq
        w2 = math.pow(w_tq, 2)
        sum += w2
    
    length = math.sqrt(sum)
    return length, query_terms_weight

    

### Now in the following methods we want to calculate documents score for a given query

In [138]:
#This method will calculate cosine score for given query and document 
# Now we want to calculate documents' lengths 


def update_doc_length_dict():
    
    doc_length_dict = {}
    for doc in doc_term_dict:

        doc_length = calculate_doc_vector_length(doc)
        doc_length_dict[doc] = doc_length
    
    return doc_length_dict

doc_length_dict = update_doc_length_dict()


In [139]:
doc_length_dict

{'1': 6.755710210411699,
 '2': 8.855494042463572,
 '3': 10.35587075668064,
 '4': 14.975150012150271,
 '5': 7.984264683540622,
 '6': 9.621271703342583,
 '7': 8.321658538301413,
 '8': 8.371825065307618,
 '9': 8.569639200082264,
 '10': 9.246151300075217}

### Now we want to calculate cosine score for all the documents and a given query 

In [140]:
#Before we start, we have to normalize the query and make a list of its terms 
def calculate_cosine_score (normalized_query_term_list):
    
    doc_score_dict = {}
    for i in range(1, number_of_all_docs + 1):
        doc_score_dict[str(i)] = 0
    
    
    query_length, query_terms_weight = calculate_query_vector_length(normalized_query_term_list)
    
    for term in normalized_query_term_list: 
        
        #Calculating weight of the term in the query 
        w_tq = query_terms_weight[term]
        
        #Fetch the postings list of the term
        postings_list = list(inverted_index_dict[term]["docs"].keys())
        
        #Updating the score of each doc inn doc_score_dict
        for doc in postings_list:
            w_td = calculate_weight(term, doc, inverted_index_dict)
            doc_score_dict[doc] += w_td * w_tq
    
    #Normalizing the scores of documents
    for doc in doc_score_dict.keys():
        doc_score_dict[doc] = doc_score_dict[doc] / (doc_length_dict[doc] * query_length)
    
    return doc_score_dict
        

In [145]:
#Now we will score documents based on query and tfidf scoring method 
def query_tfidf ():
    
    query_response = {}
    query = input()
    query_tokens = query.split()
    
    #This counter is to check whether there is any match with this query or not
    counter = 0
    
    ans = calculate_cosine_score(query_tokens)
    
    sort_ans = {k: v for k, v in sorted(ans.items(), key=lambda item: item[1])}
    
    return sort_ans
    

In [146]:
ans = query_tfidf()

گزارش تابناک


In [147]:
ans

{'2': 0.0,
 '4': 0.0,
 '6': 0.0,
 '8': 0.018737939938753122,
 '9': 0.023815886907196986,
 '3': 0.03029600483383471,
 '10': 0.03393211946471305,
 '7': 0.037701800555614236,
 '5': 0.0392949786784668,
 '1': 0.046440936738144145}